In [32]:
# ##google colab用にchainerのインストール
# !curl https://colab.chainer.org/install | sh -

In [33]:
# ##google driveのマウント
# from google.colab import drive
# drive.mount('/content/drive')

In [34]:
import numpy as np
import chainer

from chainer import cuda,Function,report,training,utils,Variable
from chainer import datasets,iterators,optimizers,serializers
from chainer import Link,Chain,ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.datasets import tuple_dataset
import matplotlib.pyplot as plt


%matplotlib inline

In [35]:
##GPU環境の設定を確認
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [36]:
##colab用にデータのpathを指定
##My Driveのカンマはpath指定の時にはいらない

#DATA_PATH = "/content/drive/My Drive/resarch"
#!ls /content/drive/'My Drive'/resarch/

DATA_PATH='./'

In [37]:
##データの読み込み
import pickle

f=open(DATA_PATH+"/train_data201604_without_normalize_2.txt","rb")
pre_pre_data=pickle.load(f)
##別に取得したデータを結合させる(201604のデータ)
f=open(DATA_PATH+"/train_data201604_without_normalize_2.txt","rb")
extend_data=pickle.load(f)


f=open(DATA_PATH+"/train_label201604_2.txt","rb")
label=pickle.load(f)
##別に取得したデータを結合させる(201604のデータ)
f=open(DATA_PATH+"train_label201604_2.txt","rb")
extend_label=pickle.load(f)

In [38]:
print("origine length: %d" % len(pre_pre_data))
print("extend length: %d" % len(extend_data))

##追加データを配列に加える処理
##特徴量
for i in range(len(extend_data)):
    pre_pre_data.append(extend_data[i])
##正解ラベル
for i in range(len(extend_label)):
    label.append(extend_label[i])

#####データの長さの確認
print("append data: %d" % len(pre_pre_data))

origine length: 133215


In [39]:
print(len(pre_pre_data))
print(len(label))
print(sum(label)/len(label))

133215
133215
0.3079232819126975


In [40]:
##全データのうちの最大値をとるindexを確認
p_data=np.array(pre_pre_data)
max_index=np.argwhere(p_data == p_data.max())
print(max_index)

[[11136    10     7]]


In [41]:
##すべて0のデータがないか確認
zero_index=[]
for i in range(len(p_data)):
    if p_data[i].max == 0:
        zero_index.append(1)
    else:
        zero_index.append(0)
sum(zero_index)

0

In [42]:
##だいたいどれくらい以上の値で全データのうちに占めるのかの計算
len(np.argwhere(p_data>140))/len(p_data)

0.09316518410088954

In [43]:
###極端な値が含まれているデータを除いて新しいデータセットを作ってみる
##ここでは、外れ値として上位約9%の値を除いたデータセットにする
pp_data=[]
pp_label=[]
for i in range(len(pre_pre_data)):
    if (np.max(pre_pre_data[i]) <=140 ):
        pp_data.append(pre_pre_data[i])
        pp_label.append(label[i])
    
    else:
        continue

In [44]:
##特徴量と正解ラベルのデータを同じindexで並べ替える
random_index=[]
for i in range(len(pp_data)):
    random_index.append(i)
    
##random_indexをシャッフルする
random_index=np.random.permutation(random_index)

##random_indexを使って、新しいデータ配列を作成する
pp_data2=[]
pp_label2=[]
for i in random_index:
    pp_data2.append(pp_data[i])
    pp_label2.append(pp_label[i])

# '''シャッフルしない場合'''
# pp_data2=pp_data
# pp_label2=pp_label

#### 各画像ごとの各列の正解ラベルに対しての合計をnumpyで格納して、各列の正解ラベルに対しての相関係数を求める

In [45]:
soukan=[]
num_sum=[]

soukan_pre_data=np.array(pp_data2)
soukan_label=np.array(pp_label2)

col0=[]
col1=[]
col2=[]
col3=[]
col4=[]
col5=[]
col6=[]
col7=[]
col8=[]
col9=[]
col10=[]

##各ユーザーごとに各列の合計を取ってくる
for i in range(len(pp_data2)):
    ##i番目のユーザーの各列の合計
    num_sum=soukan_pre_data[i].sum(axis=0)
    
    for j in range(len(num_sum)):
        
        if (j==0):
            col0.append(num_sum[j])
        if (j==1):
            col1.append(num_sum[j])
        if (j==2):
            col2.append(num_sum[j])
        if (j==3):
            col3.append(num_sum[j])
        if (j==4):
            col4.append(num_sum[j])
        if (j==5):
            col5.append(num_sum[j])
        if (j==6):
            col6.append(num_sum[j])
        if (j==7):
            col7.append(num_sum[j])
        if (j==8):
            col8.append(num_sum[j])
        if (j==9):
            col9.append(num_sum[j])
        if (j==10):
            col10.append(num_sum[j])
        
corr0=np.corrcoef(col0,soukan_label)[0][1]
corr1=np.corrcoef(col1,soukan_label)[0][1]
corr2=np.corrcoef(col2,soukan_label)[0][1]
corr3=np.corrcoef(col3,soukan_label)[0][1]
corr4=np.corrcoef(col4,soukan_label)[0][1]
corr5=np.corrcoef(col5,soukan_label)[0][1]
corr6=np.corrcoef(col6,soukan_label)[0][1]
corr7=np.corrcoef(col7,soukan_label)[0][1]
corr8=np.corrcoef(col8,soukan_label)[0][1]
corr9=np.corrcoef(col9,soukan_label)[0][1]
corr10=np.corrcoef(col10,soukan_label)[0][1]

corr_list=[corr0,corr1,corr2,corr3,corr4,corr5,corr6,corr7,corr8,corr9,corr10]

##相関の高いindex順に並べ替える
num_list=[]

for i in range(len(corr_list)):
    num=np.argsort(corr_list)[::-1][i]
    num_list.append(num)

In [46]:
###相関を左右に降る場合はここを実行する
#num_list=[5,9,8,7,0,3,4,1,10,6,2]

In [47]:
print(corr_list)
print(num_list)
##指定した順に列を並べ替える
pp_data3=[]

for i in range(len(pp_data2)):
    data=pp_data2[i][:, num_list]
    pp_data3.append(data)

[0.04196269420740168, 0.034791228451347245, 0.1780146077667481, 0.005749575040845595, 0.019707631682382095, 0.0493511843004002, 0.1170317024413279, 0.012661345003537865, 0.0659387221494236, 0.14885902511358595, 0.017949394562392463]
[2, 9, 6, 8, 5, 0, 1, 4, 10, 7, 3]


In [48]:
'''◯0以外のところを上限256として反転させる or
0のところを含めて上限256として反転させる'''
pp_reverse=[]
for i in range(len(pp_data3)):
    add_data=np.where(pp_data3[i]>0 ,256-pp_data3[i],pp_data3[i])
    pp_reverse.append(add_data)
#pp_reverse=pp_data3

In [49]:
import sys
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                      buy|    514568|
|                     buy2|     16112|
|                     col0|   1043560|
|                     col1|   1043560|
|                    col10|   1043560|
|                     col2|   1043560|
|                     col3|   1043560|
|                     col4|   1043560|
|                     col5|   1043560|
|                     col6|   1043560|
|                     col7|   1043560|
|                     col8|   1043560|
|                     col9|   1043560|
|                    label|   1080112|
|                   no_buy|    514568|
|                  no_buy2|    104816|
|                   p_data| 351687728|
|                  pp_data|   1043560|
|                 pp_data2|   1043560|
|                 pp_data3|   1043560|
|               pp_data_np|1056000128|
|                 pp_label|   1043560|
|                pp_label2|   1043560|
|               pp_revers

In [50]:
import gc
del col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,label,p_data,pp_data,pp_data2,pp_data3,pp_label,pre_pre_data,soukan_label,soukan_pre_data,zero_index
#del extend_data,extend_label
gc.collect()

4

#### テストデータだけ切り離し、標準化して保存

In [51]:
test_buy=[]
test_nobuy=[]
test_index=[]
x_test_add_noise=[]
y_test_add_noise=[]
length=int(len(pp_reverse)*0.1)
for i in range(length):
    index=int(np.random.choice(random_index,1))
    test_index.append(index)
    
for i in test_index:
    x_test_add_noise.append(pp_reverse[i])
    y_test_add_noise.append(pp_label2[i])
    
    
train_index=list(set(random_index)-set(test_index))
###正例、不例のサンプリングをする
##正例、不例のindexを取得しておく

buy=[]
no_buy=[]
for i in train_index:
    if pp_label2[i]==1:
        buy.append(i)
    else:
        no_buy.append(i)
print('positive data:{}'.format(len(buy)))
print('negative data:{}'.format(len(no_buy)))

positive data:34018
negative data:80361


In [52]:
###train
train_buy=[]
train_nobuy=[]
length=int(len(pp_reverse)*0.9*0.9)
for i in range(length):
    index=int(np.random.choice(buy,1))
    train_buy.append(index)
for i in range(length):
    index=int(np.random.choice(no_buy,1))
    train_nobuy.append(index)
    
buy2=list(set(buy)-set(train_buy))
no_buy2=list(set(no_buy)-set(train_nobuy))

##train
x_train_add_noise=[]
y_train_add_noise=[]

valid_x_test_add_noise=[]
valid_y_test_add_noise=[]

for i in train_buy:
    x_train_add_noise.append(pp_reverse[i])
    y_train_add_noise.append(pp_label2[i])
for i in train_nobuy:
    x_train_add_noise.append(pp_reverse[i])
    y_train_add_noise.append(pp_label2[i])
    
for i in buy2:
    valid_x_test_add_noise.append(pp_reverse[i])
    valid_y_test_add_noise.append(pp_label2[i])
for i in no_buy2:
    valid_x_test_add_noise.append(pp_reverse[i])
    valid_y_test_add_noise.append(pp_label2[i])

def scale(X):
    """データ行列Xを属性ごとに標準化したデータを返す"""
    # 属性の数 (列の数)
    delta=1e-3
    col = X[0].shape[1]

   
    # 属性ごとにデータを標準化or正則化
    for j in range(len(X)):
        for i in range(col):
            # 属性ごとに平均値と標準偏差を計算
            #mu = np.mean(X[j,:,[i]])
            mu = np.mean(X[j][:,[i]])
            #max_num = np.max(X[j][:,i])
            #sigma = np.std(X[j,:,[i]])
            sigma = np.std(X[j][:,[i]])
            #min_num = np.min(X[j][:,i])
            #X[j,:,[i]]= (X[j,:,[i]]- mu) / (sigma+delta)
            X[j][:,[i]]= (X[j][:,[i]]- mu) / (sigma+delta)
            #X[j][:,[i]]= (X[j][:,[i]]- min_num) / (max_num - min_num)

    return X
  
##各ユーザーデータを列ごとに標準化(列の値が全て0の場合を考えて、分母に極小の値を加えて標準化)
pp_data_np=np.asarray(x_test_add_noise)
pre_data2=scale(pp_data_np)

In [53]:
np.set_printoptions(threshold=np.inf)
len(np.unique(np.where(np.isnan(pre_data2))[0]))
nan_list=np.unique(np.where(np.isnan(pre_data2))[0])

##nanの部分を削除
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]
pre_data3=dellist(pre_data2,nan_list)
label=dellist(y_test_add_noise,nan_list)

'''特徴量:pre_data3   ラベル:label'''
##データの長さがずれないか確認
print(len(label))
print(len(pre_data3))

12641
12641


In [54]:
##test dataの保存
import pickle
f=open("./x_test_corr_high_test_without_add_some_noise_samesample_only6.txt","wb")
pickle.dump(pre_data3, f)
f=open("./y_test_corr_high_test_without_add_some_noise_samesample_only6.txt","wb")
pickle.dump(label, f)

print(sum(label)/len(label))

0.29673285341349576


In [55]:
##validation 標準化
pp_data_np=np.asarray(valid_x_test_add_noise)
pre_data2=scale(pp_data_np)

np.set_printoptions(threshold=np.inf)
len(np.unique(np.where(np.isnan(pre_data2))[0]))
nan_list=np.unique(np.where(np.isnan(pre_data2))[0])

##nanの部分を削除
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]
pre_data3=dellist(pre_data2,nan_list)
label=dellist(valid_y_test_add_noise,nan_list)

'''特徴量:pre_data3   ラベル:label'''
##データの長さがずれないか確認
print(len(label))
print(len(pre_data3))
print(sum(label)/len(label))

##valid
f=open("./validation_x_test_corr_high_test_without_add_some_noise_samesample_only6.txt","wb")
pickle.dump(pre_data3, f)
f=open("./validation_y_test_corr_high_test_without_add_some_noise_samesample_only6.txt","wb")
pickle.dump(label, f)

24265
24265
0.07216154955697507


In [56]:
###正例、不例のサンプリングをする
##正例、不例のindexを取得しておく

buy=[]
no_buy=[]
for i in range(len(y_train_add_noise)):
    if y_train_add_noise[i]==1:
        buy.append(i)
    else:
        no_buy.append(i)
print('positive data:{}'.format(len(buy)))
print('negative data:{}'.format(len(no_buy)))

positive data:102399
negative data:102399


### データ水増し

In [57]:
'''データの水増しを行う
不例を20万枚増やして正例データを同じ数になるようにしてみる
正例、不例ともにごま塩加工で増やす
'''
##塩の割合を指定
#s_vs_p = 0.7
s_vs_p = 0.7
##全体的にごま塩を増やしたいなら、ここを変える
amount = 0.15
#amount = 0.2
from skimage import transform
import random


'''正例、不例ともに画素10%にごま塩加工'''
salt_papper_posi_x=[]
salt_papper_posi_y=[]
salt_papper_nega_x=[]
salt_papper_nega_y=[]

salt_papper_posi_x2=[]
salt_papper_posi_y2=[]
salt_papper_nega_x2=[]
salt_papper_nega_y2=[]

no_buy_loop = 200000-len(no_buy)

'''負例データ'''
for j in range(no_buy_loop):
    ##データindexのランダム選択
    nega=int(np.random.choice(no_buy,1))
    src = x_train_add_noise[nega]
    sp_img = src.copy()
    # 塩モード
    num_salt = np.ceil(amount * src.size * s_vs_p)
    coords = [np.random.randint(0, i , int(num_salt)) for i in sp_img.shape]
    ##代入する値をランダムに選択(200から255)
    input_number = int(np.random.randint(116, 256 ,1))
    sp_img[coords] = input_number

    # 胡椒モード
    num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i , int(num_pepper)) for i in sp_img.shape]
    sp_img[coords] = 0
    
    salt_papper_nega_x.append(sp_img)
    salt_papper_nega_y.append(y_train_add_noise[nega])
    


buy_loop = 200000-len(buy)

'''正例データ'''
for j in range(buy_loop):
    ##データindexのランダム選択
    posi=int(np.random.choice(buy,1))
    src = x_train_add_noise[posi]
    sp_img = src.copy()
    # 塩モード
    num_salt = np.ceil(amount * src.size * s_vs_p)
    coords = [np.random.randint(0, i , int(num_salt)) for i in sp_img.shape]
    ##代入する値をランダムに選択(200から255)
    input_number = int(np.random.randint(116, 256 ,1))
    sp_img[coords] = input_number

    # 胡椒モード
    num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i , int(num_pepper)) for i in sp_img.shape]
    sp_img[coords] = 0
    
    salt_papper_posi_x.append(sp_img)
    salt_papper_posi_y.append(y_train_add_noise[posi])
    
# '''違う値の塩の代入パターン1'''

# no_buy_loop = 100000-len(no_buy)
# s_vs_p = 0.7
# amount = 0.15
# nega_loop=200000
# '''負例データ'''
# for j in range(no_buy_loop):
#     ##データindexのランダム選択
#     nega=int(np.random.choice(no_buy,1))
#     src = valid_x_train_add_noise[nega]
#     sp_img = src.copy()
#     # 塩モード
#     num_salt = np.ceil(amount * src.size * s_vs_p)
#     coords = [np.random.randint(0, i , int(num_salt)) for i in sp_img.shape]
#     ##代入する値をランダムに選択(115, 151)
#     input_number = int(np.random.randint(116, 256 ,1))
#     sp_img[coords] = input_number

#     # 胡椒モード
#     num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
#     coords = [np.random.randint(0, i , int(num_pepper)) for i in sp_img.shape]
#     sp_img[coords] = 0
    
#     salt_papper_nega_x2.append(sp_img)
#     salt_papper_nega_y2.append(valid_y_train_add_noise[nega])


# buy_loop = 100000-len(buy)
# posi_loop=200000
# '''正例データ'''
# for j in range(buy_loop):
#     ##データindexのランダム選択
#     posi=int(np.random.choice(buy,1))
#     src = valid_x_train_add_noise[posi]
#     sp_img = src.copy()
#     # 塩モード
#     num_salt = np.ceil(amount * src.size * s_vs_p)
#     coords = [np.random.randint(0, i , int(num_salt)) for i in sp_img.shape]
#     ##代入する値をランダムに選択(115, 151)
#     input_number = int(np.random.randint(116, 256 ,1))
#     sp_img[coords] = input_number

#     # 胡椒モード
#     num_pepper = np.ceil(amount* src.size * (1. - s_vs_p))
#     coords = [np.random.randint(0, i , int(num_pepper)) for i in sp_img.shape]
#     sp_img[coords] = 0
    
#     salt_papper_posi_x2.append(sp_img)
#     salt_papper_posi_y2.append(valid_y_train_add_noise[posi])





/home/shin/py3env/lib/python3.5/site-packages/ipykernel_launcher.py:39: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/shin/py3env/lib/python3.5/site-packages/ipykernel_launcher.py:44: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/shin/py3env/lib/python3.5/site-packages/ipykernel_launcher.py:64: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in 

In [58]:
###データappend
for i in range(len(salt_papper_posi_x)):
    x_train_add_noise.append(salt_papper_posi_x[i])
    y_train_add_noise.append(salt_papper_posi_y[i])
    
for i in range(len(salt_papper_nega_x)):
    x_train_add_noise.append(salt_papper_nega_x[i])
    y_train_add_noise.append(salt_papper_nega_y[i])
    
# for i in range(len(salt_papper_posi_x2)):
#     valid_x_train_add_noise.append(salt_papper_posi_x2[i])
#     valid_y_train_add_noise.append(salt_papper_posi_y2[i])
    
# for i in range(len(salt_papper_nega_x2)):
#     valid_x_train_add_noise.append(salt_papper_nega_x2[i])
#     valid_y_train_add_noise.append(salt_papper_nega_y2[i])

'''平滑化処理'''
# ##正例、不例のindexを更新

# buy=[]
# no_buy=[]
# for i in range(len(y_train_add_noise)):
#     if y_train_add_noise[i]==1:
#         buy.append(i)
#     else:
#         no_buy.append(i)


# import cv2
# average_square = (3,3)

# nega_loop=50000
# posi_loop=50000
# '''負例データ'''
# for j in range(nega_loop):
#     ##データindexのランダム選択
#     nega=int(np.random.choice(no_buy,1))
#     src = x_train_add_noise[nega]
#     sp_img = src.copy()
#     ##平滑化
#     blur_img = cv2.blur(sp_img, average_square)
    
#     salt_papper_nega_x2.append(blur_img)
#     salt_papper_nega_y2.append(y_train_add_noise[nega])

# '''正例データ'''
# for j in range(posi_loop):
#     ##データindexのランダム選択
#     posi=int(np.random.choice(buy,1))
#     src = x_train_add_noise[posi]
#     sp_img = src.copy()
#     ##平滑化
#     blur_img = cv2.blur(sp_img, average_square)
    
#     salt_papper_posi_x2.append(blur_img)
#     salt_papper_posi_y2.append(y_train_add_noise[posi])

    
# for i in range(len(salt_papper_posi_x2)):
#     x_train_add_noise.append(salt_papper_posi_x2[i])
#     y_train_add_noise.append(salt_papper_posi_y2[i])
    
# for i in range(len(salt_papper_nega_x2)):
#     x_train_add_noise.append(salt_papper_nega_x2[i])
#     y_train_add_noise.append(salt_papper_nega_y2[i])

'平滑化処理'

### 標準化

In [59]:

def scale(X):
    """データ行列Xを属性ごとに標準化したデータを返す"""
    # 属性の数 (列の数)
    delta=1e-3
    col = X[0].shape[1]

   
    # 属性ごとにデータを標準化or正則化
    for j in range(len(X)):
        for i in range(col):
            # 属性ごとに平均値と標準偏差を計算
            #mu = np.mean(X[j,:,[i]])
            mu = np.mean(X[j][:,[i]])
            #max_num = np.max(X[j][:,i])
            #sigma = np.std(X[j,:,[i]])
            sigma = np.std(X[j][:,[i]])
            #min_num = np.min(X[j][:,i])
            #X[j,:,[i]]= (X[j,:,[i]]- mu) / (sigma+delta)
            X[j][:,[i]]= (X[j][:,[i]]- mu) / (sigma+delta)
            #X[j][:,[i]]= (X[j][:,[i]]- min_num) / (max_num - min_num)

    return X
  
##各ユーザーデータを列ごとに標準化(列の値が全て0の場合を考えて、分母に極小の値を加えて標準化)
pp_data_np=np.asarray(x_train_add_noise)
pre_data2=scale(pp_data_np)

# '''全データの最大値で正規化'''
# pre_data2=[]
# for i in range(len(pp_data2)):
#     divided=pp_data2[i]/255
#     pre_data2.append(divided)

すべてのログが0のものも取り出しているから、削除する。<br>
とりあえずnanのあるところはindexを取得して、削除する。<br>
そのindexを参照して、正解ラベルも削除

In [60]:
np.set_printoptions(threshold=np.inf)
len(np.unique(np.where(np.isnan(pre_data2))[0]))
nan_list=np.unique(np.where(np.isnan(pre_data2))[0])

##nanの部分を削除
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]
pre_data3=dellist(pre_data2,nan_list)
label=dellist(y_train_add_noise,nan_list)

'''特徴量:pre_data3   ラベル:label'''
##データの長さがずれないか確認
print(len(label))
print(len(pre_data3))

400000
400000


In [61]:
##train保存
##train
f = open('./validation_x_train_corr_high_test_without_add_some_noise_samesample_only6.txt', 'wb')
pickle.dump(pre_data3, f)
f = open('./validation_y_train_corr_high_test_without_add_some_noise_samesample_only6.txt', 'wb')
pickle.dump(label, f)


In [62]:
print(sum(label)/len(label))

0.5
